# The Actor-Critic Method
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/matyama/deep-rl-hands-on/blob/main/12_a2c.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
        Run in Google Colab
    </a>
  </td>
</table>

## Variance Reduction
Let's start by recalling the policy gradient defined by the *Policy Gradients (PG)* method:
$$
\nabla J \approx \mathbb{E}[Q(s, a) \nabla \log(\pi(a|s))]
$$

One of the weak points of the PG method is that the gradient scales $Q(s, a)$ may experience quite significant variance* which does not help the training at all. We fixed this issue by introducing a fixed *baseline* value (e.g. mean reward) that was subtracted from the gradient scales Q.

\* Recall formal defintion: $\mathbb{V}[X] = \mathbb{E}[(X - \mathbb{E}[X])^2]$

Let's illustrate this problem and solution on simple example:
* Assume there are three actions with $Q_1$, $Q_2$ some small positive values and $Q_3$ being large negative
* In this case there will be small positive gradient towards fist two actions and large negative one repelling the policy from the third one
* Now imagine $Q_1$ and $Q_2$ were large positive values instead. Then $Q_3$ would become small but positive value. The gradient would still push the policy towards fist two actions but it would direct the gradient towards the trird one a bit as well (instead of pushing it away from it)!

Now it's a bit more clear why subtracting a constant value that we called the *baseline* helps.